In [ ]:
pip install Flask Flask-RESTful flask-httpauth pyodbc

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\amitk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
pip install flasgger

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
     -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
     ------------------ --------------------- 1.8/4.0 MB 7.2 MB/s eta 0:00:01
     --------------------- ------------------ 2.1/4.0 MB 7.8 MB/s eta 0:00:01
     --------------------- ------------------ 2.1/4.0 MB 7.8 MB/s eta 0:00:01
     --------------------- ------------------ 2.1/4.0 MB 7.8 MB/s eta 0:00:01
     --------------------- ------------------ 2.1/4.0 MB 7.8 MB/s eta 0:00:01
     --------------------- ------------------ 2.1/4.0 MB 7.8 MB/s eta 0:00:01
     --------------------- ------------------ 2.1/4.0 MB 7.8 MB/s eta 0:00:01
     --------------------- ------------------ 2.1/4.0 MB 7.8 MB/s eta 0:00:01
     --------------------- ------------------ 2.1/4.0 MB 7.8 MB/s eta 0:00:01
     --


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\amitk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
"""
Flask-RESTful API with PostgreSQL (pyodbc) + Swagger UI

Features:
- All HTTP Methods: GET, POST, PUT, PATCH, DELETE, OPTIONS
- Database connection via pyodbc
- Basic Auth for protected methods
- Swagger UI documentation
- Compatible with Jupyter (disables Flask reloader)
"""

from flask import Flask, request, make_response
from flask_restful import Resource, Api
from flask_httpauth import HTTPBasicAuth
from werkzeug.security import generate_password_hash, check_password_hash
import pyodbc
from flasgger import Swagger

app = Flask(__name__)
api = Api(app)
Swagger(app)
auth = HTTPBasicAuth()

# ------------------- Authentication -------------------
users = {"admin": generate_password_hash("secret")}

@auth.verify_password
def verify_password(username, password):
    if username in users and check_password_hash(users.get(username), password):
        return username
    return None

# ------------------- Database Connection -------------------
DB_DRIVER = '{PostgreSQL ODBC Driver(UNICODE)}'
DB_SERVER = 'localhost'
DB_PORT = 5432
DB_DATABASE = 'amba'
DB_UID = 'postgres'
DB_PWD = 'amit@123'

def get_db_connection():
    conn_str = f'DRIVER={DB_DRIVER};SERVER={DB_SERVER};PORT={DB_PORT};DATABASE={DB_DATABASE};UID={DB_UID};PWD={DB_PWD}'
    return pyodbc.connect(conn_str)

# ------------------- Resource -------------------
class ItemResource(Resource):

    def get(self):
        """
        Get the latest item
        ---
        responses:
          200:
            description: Returns the last inserted item
        """
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT id, message, value FROM items ORDER BY id DESC LIMIT 1")
        row = cursor.fetchone()
        cursor.close()
        conn.close()
        if row:
            item = {"id": row[0], "message": row[1], "value": float(row[2])}
        else:
            item = {}
        return {"method": "GET", "data": item}, 200

    @auth.login_required
    def post(self):
        """
        Insert a new item
        ---
        parameters:
          - in: body
            name: body
            schema:
              type: object
              properties:
                message:
                  type: string
                value:
                  type: number
              required:
                - message
        responses:
          201:
            description: Item created
        """
        data = request.get_json(force=True)
        message = data.get("message", "")
        value = data.get("value", 0)
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("INSERT INTO items (message, value) VALUES (?, ?)", (message, value))
        cursor.execute("SELECT MAX(id) FROM items")
        new_id = cursor.fetchone()[0]
        conn.commit()
        cursor.close()
        conn.close()
        return {"method": "POST", "id": new_id, "message": message, "value": value}, 201

    @auth.login_required
    def put(self):
        """
        Replace an item
        ---
        parameters:
          - in: body
            name: body
            schema:
              type: object
              properties:
                id:
                  type: integer
                message:
                  type: string
                value:
                  type: number
              required:
                - id
                - message
        responses:
          200:
            description: Item replaced
        """
        data = request.get_json(force=True)
        item_id = data.get("id")
        if not item_id:
            return {"error": "id is required for PUT"}, 400
        message = data.get("message", "")
        value = data.get("value", 0)
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("UPDATE items SET message=?, value=? WHERE id=?", (message, value, item_id))
        conn.commit()
        cursor.close()
        conn.close()
        return {"method": "PUT", "id": item_id, "message": message, "value": value}, 200

    @auth.login_required
    def patch(self):
        """
        Partial update an item
        ---
        parameters:
          - in: body
            name: body
            schema:
              type: object
              properties:
                id:
                  type: integer
                message:
                  type: string
                value:
                  type: number
              required:
                - id
        responses:
          200:
            description: Item partially updated
        """
        data = request.get_json(force=True)
        item_id = data.get("id")
        if not item_id:
            return {"error": "id is required for PATCH"}, 400
        updates = []
        params = []
        for key in ["message", "value"]:
            if key in data:
                updates.append(f"{key} = ?")
                params.append(data[key])
        if not updates:
            return {"error": "No fields to update"}, 400
        params.append(item_id)
        sql = f"UPDATE items SET {', '.join(updates)} WHERE id = ?"
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute(sql, params)
        conn.commit()
        cursor.close()
        conn.close()
        return {"method": "PATCH", "id": item_id}, 200

    @auth.login_required
    def delete(self):
        """
        Delete an item
        ---
        parameters:
          - in: body
            name: body
            schema:
              type: object
              properties:
                id:
                  type: integer
              required:
                - id
        responses:
          200:
            description: Item deleted
        """
        data = request.get_json(force=True)
        item_id = data.get("id")
        if not item_id:
            return {"error": "id is required for DELETE"}, 400
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("DELETE FROM items WHERE id=?", (item_id,))
        conn.commit()
        cursor.close()
        conn.close()
        return {"method": "DELETE", "id": item_id, "status": "deleted"}, 200

    def options(self):
        response = make_response("", 204)
        response.headers['Allow'] = 'GET,POST,PUT,PATCH,DELETE,OPTIONS'
        return response

api.add_resource(ItemResource, '/item')

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
